In [2]:
#two files will be analyzed,
#Eeloo.wld: A well-traversed expert mode small map with
#* Corruption
#* Copper, Iron, Tungsten, Gold, Palladium, Mythril, and Adamantite
#* All bosses except Duke Fishron defeated
#* All NPCs (including Santa)
#BigWorld.wld: A mostly new normal mode large map with
# * Crimson
# * Tin, Lead, ???
# * Only the guide and angler NPCs
# * Some interesting objects placed in the upper left-hand corner
#   of the map going downward, including a chest (containing , 
#   a chlorophyte bar dirt, silk, and other objects), gold coin stack, 
#   silver coin stack, mushroom work bench, wiring (yellow (1), blue(3), 
#   and red(2); many palm platforms; a sign with 2323232323 written on
#   it
#open relevant files
import struct
filename1 = 'Eeloo.wld'
filename2 = 'BigWorld.wld'
f = open(filename1,'rb')
g = open(filename2,'rb')

In [42]:
#define functions to read bytes

def seeker(f):
    """decorator to allow easy seeking"""
    def decorated(*args, **kwargs):
        if kwargs.get('seek',None) is not None:
            args[0].seek(kwargs['seek'])
        return f(*args)
    return decorated

@seeker
def int32(obj):
    data = obj.read(4)
    return struct.unpack("<I", data)[0]

@seeker
def int16(obj):
    data = obj.read(2)
    return struct.unpack("<H", data)[0]

@seeker
def byte(obj):
    data = obj.read(1)
    return struct.unpack("<B",data)[0]

@seeker
def string(obj, length):
    data = obj.read(length)
    return struct.unpack("<%ds" %length,data)

@seeker
def hexstring(obj, length):
    data = obj.read(length)
    return struct.unpack("<%ds" %length,data)[0].encode('hex')

@seeker
def int64(obj):
    data = obj.read(8)
    return struct.unpack("<Q", data)[0]

@seeker
def float32(obj):
    data = obj.read(4)
    return struct.unpack("<f", data)[0]

@seeker
def float64(obj):
    data = obj.read(8)
    return struct.unpack("<d", data)[0]

@seeker
def char(obj):
    data = obj.read(1)
    return struct.unpack('<c', data)[0]



In [17]:
(version1, version2) = (int32(f,seek=0),int32(g,seek=0))
print version1, version2

188 188


In [18]:
(namesize1, namesize2) = (byte(f,seek=0x79), byte(g,seek=0x79))
print namesize1, namesize2
#59 59 is a strange result, but it's consistent...

59 59


In [19]:
(name1, name2) = (string(f,namesize1, seek=0x7A),
                  string(g,namesize2, seek=0x7A))
print name1
print name2
#okay, so both of these have the name contained here,
#and they start at the same position (6 after 0x7A, or 0x80)
#the world ID is supposed to come after it (4 bytes), and the name
#is incorporated into the .map filenames under .plr files, which, for
#a particular character for these two maps is
#ed74bf72-911f-471b-82b9-667b587ff83d (Eeloo)
#3988c36c-42c5-46a6-819f-a5b36b41f0a9 (BigWorld)
#let's try converting the names to Hex

('\xf8?\xf0\xe3\x07\x05Eeloo\n1441374647\x01\x00\x00\x00\xbb\x00\x00\x00r\xbft\xed\x1f\x91\x1bG\x82\xb9f{X\x7f\xf8=K$JG\x00\x00\x00\x00\x80\x06\x01\x00\x00',)
('\xf8?\xf0\xe3\x07\x08BigWorld\x0884852984\x01\x00\x00\x00\xbc\x00\x00\x00l\xc3\x889\xc5B\xa6F\x81\x9f\xa5\xb3kA\xf0\xa9\xd7=R^\x00\x00\x00\x00\x00\r\x02\x00',)


In [20]:
print name1[0].encode('hex')
print name2[0].encode('hex')
#the IDs can be seen to be representative of the filenames above, but
#the IDs are split into two parts with different endianness for them
#the first 4 bytes are reversed for the first part
#the next 2 bytes are reversed for the second part
#the next 2 bytes are reversed for the third part
#the next 2 bytes are in order for the fourth part
#the next 6 bytes are in order for the fifth part
#ergo, the map ID for this version is 16 bytes,
#the .map filename maker reverses the first 8
#the location of the map ID is 30 bytes after 0x80 for Eeloo, and
#31 bytes after 0x80 for BigWorld; this is concerning because
#BigWorld is 3 more bytes in name length, and only differences of  
#0 (indicating padded data) and 3 bytes would make sense...
#maybe there is an issue with the different world types and a third/fourth 
#should be included for reference

f83ff0e3070545656c6f6f0a3134343133373436343701000000bb00000072bf74ed1f911b4782b9667b587ff83d4b244a47000000008006010000
f83ff0e30708426967576f726c6408383438353239383401000000bc0000006cc38839c542a646819fa5b36b41f0a9d73d525e00000000000d0200


In [21]:
filename3 = 'Duna.wld'
filename4 = 'Jool.wld'#yes, this is a KSP reference
h = open(filename3,'rb')
j = open(filename4,'rb')
#Duna is a large, pre-hardmode world with a small amount of building
#and a merchant NPC (also has copper)
#Jool is a large, pre-hardmode world that is untouched and is in 
#expert mode

In [23]:
(namesize3, namesize4) = (byte(h,seek=0x79), byte(j,seek=0x79))
print namesize1, namesize2

59 59


In [24]:
(name3, name4) = (string(h,namesize3, seek=0x7A),
                  string(j,namesize4, seek=0x7A))
print name3
print name4

('\xf8?\xf0\xe3\x07\x04Duna\n2116190617\x01\x00\x00\x00\xbc\x00\x00\x00\xfa&\xe2/\x9a\x8f\xe3N\xb3\xd51\x08D\xdd\xd8]`jk\x0e\x00\x00\x00\x00\x00\r\x02\x00\x00\x00',)
('\xf8?\xf0\xe3\x07\x04Jool\n1983026486\x01\x00\x00\x00\xbc\x00\x00\x00[\x88\x1e\xd5\xc3\xe2\x14B\x9d\xb9R\xaf9e\xd9)\x04\xe8\xe5Q\x00\x00\x00\x00\x00\r\x02\x00\x00\x00',)


In [25]:
print name3[0].encode('hex')
print name4[0].encode('hex')
#both of these have the IDs 29 bytes after 0x7A...
#this is consistent with Eeloo's being 30 (since the filename is 1
#byte longer), but BigWorld (which was generated most recently)
#is still a mystery...let's add 1 last file

f83ff0e3070444756e610a3231313631393036313701000000bc000000fa26e22f9a8fe34eb3d5310844ddd85d606a6b0e00000000000d02000000
f83ff0e307044a6f6f6c0a3139383330323634383601000000bc0000005b881ed5c3e214429db952af3965d92904e8e55100000000000d02000000


In [26]:
filename5 = 'MediumWorld.wld'
k = open(filename5,'rb')
#MediumWorld is a mostly untouched medium-sized crimson world that
#only has wall of flesh defeated (so it's hardmode)
#it has tin, lead, and silver

In [28]:
name5 = string(k,59, seek=0x7A)
print name5
print name5[0].encode('hex')
#ID starts 36 bytes after 0x7A, consistent with everything but BigWorld

('\xf8?\xf0\xe3\x07\x0bMediumWorld\n1013350376\x01\x00\x00\x00\xbc\x00\x00\x00"=\xf9\x7f\xe6\xbf\x9dG\x83\x8a\xadB\xde\x82J\xff|y\x0fp\x00\x00\x00',)
f83ff0e3070b4d656469756d576f726c640a3130313333353033373601000000bc000000223df97fe6bf9d47838aad42de824aff7c790f70000000


In [41]:
#in order to analyze all of these better, 
#make a function that wraps f,g,h,j, and k in functions with the same 
#parameters or a parameter list

def check_args(args, index):
    args = list(args)
    for i in range(len(args)):
        if isinstance(args[i],list):
            args[i] = args[i][index]
    return args

def check_kwargs(kwargs, index):
    for key in kwargs.keys():
        if isinstance(kwargs[key],list):
            kwargs[key] = kwargs[key][index]
    return kwargs

def apply_all(func,*args,**kwargs):
    data_list = (f, g, h, j, k)
    results = []
    for i, element in enumerate(data_list):
        newargs = [element] + check_args(args, i)
        kwargs = check_kwargs(kwargs, i)
        result = func(*newargs, **kwargs)
        results.append(result)
    return results

#test
apply_all(string,59,seek=0x7A)

[('\xf8?\xf0\xe3\x07\x05Eeloo\n1441374647\x01\x00\x00\x00\xbb\x00\x00\x00r\xbft\xed\x1f\x91\x1bG\x82\xb9f{X\x7f\xf8=K$JG\x00\x00\x00\x00\x80\x06\x01\x00\x00',),
 ('\xf8?\xf0\xe3\x07\x08BigWorld\x0884852984\x01\x00\x00\x00\xbc\x00\x00\x00l\xc3\x889\xc5B\xa6F\x81\x9f\xa5\xb3kA\xf0\xa9\xd7=R^\x00\x00\x00\x00\x00\r\x02\x00',),
 ('\xf8?\xf0\xe3\x07\x04Duna\n2116190617\x01\x00\x00\x00\xbc\x00\x00\x00\xfa&\xe2/\x9a\x8f\xe3N\xb3\xd51\x08D\xdd\xd8]`jk\x0e\x00\x00\x00\x00\x00\r\x02\x00\x00\x00',),
 ('\xf8?\xf0\xe3\x07\x04Jool\n1983026486\x01\x00\x00\x00\xbc\x00\x00\x00[\x88\x1e\xd5\xc3\xe2\x14B\x9d\xb9R\xaf9e\xd9)\x04\xe8\xe5Q\x00\x00\x00\x00\x00\r\x02\x00\x00\x00',),
 ('\xf8?\xf0\xe3\x07\x0bMediumWorld\n1013350376\x01\x00\x00\x00\xbc\x00\x00\x00"=\xf9\x7f\xe6\xbf\x9dG\x83\x8a\xadB\xde\x82J\xff|y\x0fp\x00\x00\x00',)]

In [43]:
apply_all(hexstring,59,seek=0x7A)

['f83ff0e3070545656c6f6f0a3134343133373436343701000000bb00000072bf74ed1f911b4782b9667b587ff83d4b244a47000000008006010000',
 'f83ff0e30708426967576f726c6408383438353239383401000000bc0000006cc38839c542a646819fa5b36b41f0a9d73d525e00000000000d0200',
 'f83ff0e3070444756e610a3231313631393036313701000000bc000000fa26e22f9a8fe34eb3d5310844ddd85d606a6b0e00000000000d02000000',
 'f83ff0e307044a6f6f6c0a3139383330323634383601000000bc0000005b881ed5c3e214429db952af3965d92904e8e55100000000000d02000000',
 'f83ff0e3070b4d656469756d576f726c640a3130313333353033373601000000bc000000223df97fe6bf9d47838aad42de824aff7c790f70000000']